In [23]:
import mysql.connector
from tabulate import tabulate

# Database connection
connect = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="classicmodels"
)
res = connect.cursor()

def menu():
    print("Welcome to Chennai Kulfi")
    print("Menu")
    print("Cornatto")
    print("Kulfi King")
    print("Stick Cream")
    print("Casata")
    print("Cup")

def get_price_and_quantity(ice_cream_type, ice_flavour):
    query = "SELECT price, quantity FROM product_details WHERE type_of_ice_cream = %s AND ice_flavour = %s"
    res.execute(query, (ice_cream_type, ice_flavour))
    result = res.fetchone()
    if result:
        return result[0], result[1] 
    else:
        return None, None

def order():
    ice_cream_types = {
        "cornatto": ["vanilla", "chocolate", "strawberry"],
        "kulfi king": ["pista", "badam", "kesar"],
        "stick cream": ["mango", "orange", "grape"],
        "casata": ["butterscotch", "chocolate", "vanilla"],
        "cup": ["vanilla", "chocolate", "strawberry"]
    }

    customer_name = input("Enter the customer name: ").strip()
    type_of_ice_cream = input("Select type of ice cream (cornatto/kulfi king/stick cream/casata/cup): ").strip().lower()

    if type_of_ice_cream not in ice_cream_types:
        print("Invalid ice cream type selected.")
        return

    available_flavors = ice_cream_types[type_of_ice_cream]
    ice_flavour = input(f"Enter the ice cream flavor {"/".join (available_flavors)}").strip().lower()

    if ice_flavour not in available_flavors:
        print("Invalid ice cream flavor selected.")
        return

    price, available_quantity = get_price_and_quantity(type_of_ice_cream, ice_flavour)
    if price is None:
        print("Selected ice cream type or flavor is not available.")
        return

    quantity = int(input("Enter the quantity: "))
    
    if quantity > available_quantity:
        print(f"Only {available_quantity} units available. Cannot place the order.")
        return

    amount = quantity * price
    query = """
    INSERT INTO order_details (customer_name, type_of_ice_cream, ice_flavour, quantity, per_quantity_price, amount)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    values = (customer_name, type_of_ice_cream, ice_flavour, quantity, price, amount)
    res.execute(query, values)

    new_quantity = available_quantity - quantity
    update_query = "UPDATE product_details SET quantity = %s WHERE type_of_ice_cream = %s AND ice_flavour = %s"
    res.execute(update_query, (new_quantity, type_of_ice_cream, ice_flavour))

    connect.commit()
    print("Order placed successfully.")

def show():
    query = "SELECT * FROM order_details"
    res.execute(query)
    result = res.fetchall()
    print(tabulate(result, headers=["order_id", "customer_name", "type_of_ice_cream", "ice_flavour", "quantity", "per_quantity_price", "amount", "order_date"]))

menu()
order()
show()


Welcome to Chennai Kulfi
Menu
Cornatto
Kulfi King
Stick Cream
Casata
Cup


Enter the customer name:  karthick
Select type of ice cream (cornatto/kulfi king/stick cream/casata/cup):  cup
Enter the ice cream flavor vanilla/chocolate/strawberry vanilla
Enter the quantity:  2


Order placed successfully.
  order_id  customer_name    type_of_ice_cream    ice_flavour      quantity    per_quantity_price    amount  order_date
----------  ---------------  -------------------  -------------  ----------  --------------------  --------  -------------------
         1  karthick         cornatto             vanilla                 2                    50       100  2024-08-01 16:19:52
         2  karthick         cup                  vanilla                 2                    40        80  2024-08-01 16:29:31


In [ ]:
{'/'.join(available_flavors)}): "